In [1]:
%load_ext autoreload
%autoreload 2

# Add the project directory to our module path
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# Import the rest of our modules
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

%matplotlib inline

# Import py files
from src import cm_class_LRM as LRM # Logistic Regression Model Class
from src import cm_class_KNN as KNN # K-Nearest Neighbor Model Class
from src import cm_functions_balancing as cmbal
from src import cm_functions_preprocessing as cmpre
from src import cm_functions_tuning as cmtune

# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Classifier Libraries
from sklearn.linear_model import LogisticRegression, RidgeCV, LassoCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier

# Other Libraries
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, cross_val_score
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import make_scorer, precision_score
from collections import Counter
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.utils import resample

# Create paths to folders in repo
data_folder = '../../data/'
src_folder = '../../src/'

**Import Data**

In [8]:
df=pd.read_csv('../../data/initial_clean_lc', index_col=[0])
df.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,KS,128,415,False,True,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,False,True,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,False,False,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,True,False,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,True,False,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


**Dummy `state`**

In [9]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
# create the dummy matrix
st_matrix = ohe.fit_transform(df[['state']])
# make it a pandas df
df_states = pd.DataFrame(st_matrix)
# set the column labels
df_states.columns= list(ohe.categories_[0])
# join the dataframes and drop the original column
df_new = df.join(df_states)
df_new.drop('state', axis=1, inplace=True)

In [10]:
df_new['total_charge'] = df.total_day_charge+df.total_eve_charge+df.total_night_charge+df.total_intl_charge

In [11]:
df_new = df_new.drop("area_code", axis=1)
df_new.voice_mail_plan.replace((True, False), (1, 0), inplace = True)
df_new.international_plan.replace((True, False), (1, 0), inplace = True)
df_new.churn.replace((True, False), (1, 0), inplace = True)
df_new

,account_length,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,total_charge
0,128,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.56
1,107,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.24
2,137,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.29
3,84,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.80
4,75,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,0,1,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.10
3329,68,0,0,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,63.53
3330,28,0,0,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.74
3331,184,1,0,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.53


In [12]:
df_new.to_csv(data_folder+'expanded_dataset_cm')

# MODEL 1: Default Parameters

- models:
    - DecisionTreeClassifier() 
    - KNeighborsClassifier()
    - RandomForestClassifier()
    - AdaBoostClassifier(),
    - GradientBoostingClassifier()
- scaling: StandardScaler()
- balancing: SMOTE

In [14]:
y = df_new.churn
X = df_new.drop("churn", axis=1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size = .25)

In [16]:
# create kfolds object
kf = KFold(n_splits = 5, shuffle=True, random_state = 42)

In [17]:
for train_ind, val_ind in kf.split(X_train, y_train):

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])
    mu_performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.ss_scale(X_t, X_val)

    # instantiate and fit SMOTE:
    smote = SMOTE(random_state = 42)
    X_t_sc_bal, y_t_bal = smote.fit_resample(X_t_sc, y_t)
    
    classifiers = [DecisionTreeClassifier(), KNeighborsClassifier(),
               RandomForestClassifier(),AdaBoostClassifier(),
               GradientBoostingClassifier()]

    for clf in classifiers:
        train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, 
                                   estimator=clf, scoring=recall,cv=10)

        # Predict
        y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

        conf_matrix = confusion_matrix(y_val,y_pred)

        # Store results
        performance.loc[clf.__class__.__name__+'_default',
                        ['Train_Recall','Test_Recall','Test_Specificity']] = [
            train_cv.mean(),
            recall_score(y_val,y_pred),
            conf_matrix[0,0]/conf_matrix[0,:].sum()
        ]
    mu_performance = pd.concat([mu_performance,performance])
    print(mu_performance)


                                   Train_Recall Test_Recall Test_Specificity
DecisionTreeClassifier_default         0.960519      0.8625         0.961905
KNeighborsClassifier_default            0.98199      0.4875          0.77381
RandomForestClassifier_default         0.935542        0.75         0.959524
AdaBoostClassifier_default             0.893161      0.5875         0.911905
GradientBoostingClassifier_default     0.944257      0.8125         0.990476
                                   Train_Recall Test_Recall Test_Specificity
DecisionTreeClassifier_default         0.958044        0.84         0.945882
KNeighborsClassifier_default           0.975551    0.573333         0.731765
RandomForestClassifier_default         0.949905         0.8         0.967059
AdaBoostClassifier_default             0.886455        0.76         0.917647
GradientBoostingClassifier_default     0.952237        0.84         0.990588
                                   Train_Recall Test_Recall Test_Specificity

# MODEL 2B: GBC w/ MinMax Scaling

- model:
    - GradientBoostingClassifier()
- scaling: MinMaxScaler()
- balancing: SMOTE

In [22]:
for train_ind, val_ind in kf.split(X_train, y_train):

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.mm_scale(X_t, X_val)

    # instantiate and fit SMOTE:
    smote = SMOTE(random_state = 42)
    X_t_sc_bal, y_t_bal = smote.fit_resample(X_t_sc, y_t)
    
    clf = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.2,  
        random_state = 15)

    train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, estimator=clf, scoring=recall,cv=10)

    # Predict
    y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

    conf_matrix = confusion_matrix(y_val,y_pred)

    # Store results
    performance.loc['GBC_exp_lr0.2_mm',
                    ['Train_Recall','Test_Recall','Test_Specificity']] = [
        train_cv.mean(),
        recall_score(y_val,y_pred),
        conf_matrix[0,0]/conf_matrix[0,:].sum()
    ]
    print(performance)

                 Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.2_mm     0.960529         0.8         0.985714
                 Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.2_mm     0.955736        0.84                1
                 Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.2_mm     0.957881    0.838235         0.986111
                 Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.2_mm     0.958544    0.885714         0.986047
                 Train_Recall Test_Recall Test_Specificity
GBC_exp_lr0.2_mm     0.950819    0.861538         0.995392


# Hyperparameter Tuning Summary of results

- methods:
    - GridSearchCV
    - Trial and Error

In [19]:
# EXAMPLE GridSearchCV
X_train_ss, X_test_ss = cmpre.ss_scale(X_train, X_test)

smote = SMOTE(random_state = 15)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ss, y_train)

param_grid = {
    "loss":["exponential", "deviance"],
    "learning_rate": [0.01],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    }

clf = GradientBoostingClassifier(random_state=15)

search = GridSearchCV(estimator=clf,param_grid=param_grid,n_jobs=-1,scoring='recall')

# Train search object
search.fit(X_train_resampled, y_train_resampled)

# Get best estimator
best = search.best_estimator_
print('Best parameters: \n\n',search.best_params_,'\n')

Best parameters: 

 {'learning_rate': 0.01, 'loss': 'exponential'} 



### **Results:**
- Initial best parameter summary: 

 {'loss': 'exponential', 'learning_rate': 0.01, 'min_samples_leaf': 0.1, 'min_samples_leaf': 0.1, 'criterion': 'mae', 'max_depth': 3} 

# MODEL 3A: GBC w/ hyperparameters (Best So Far)

## *LEANA, RESULTS FOR THIS MODEL CAN BE FOUND IN 06_cm_modeling 'Model 2, Using X2'*

- model:
    - GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)
- scaling: StandardScaler()
- balancing: SMOTE

In [20]:
for train_ind, val_ind in kf.split(X_train, y_train):

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.ss_scale(X_t, X_val)

    # instantiate and fit SMOTE:
    smote = SMOTE(random_state = 42)
    X_t_sc_bal, y_t_bal = smote.fit_resample(X_t_sc, y_t)
    
    clf = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)

    train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, estimator=clf, scoring=recall,cv=10)

    # Predict
    y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

    conf_matrix = confusion_matrix(y_val,y_pred)

    # Store results
    performance.loc['GBC_exp_maxd3_mae_ss',
                    ['Train_Recall','Test_Recall','Test_Specificity']] = [
        train_cv.mean(),
        recall_score(y_val,y_pred),
        conf_matrix[0,0]/conf_matrix[0,:].sum()
    ]
    print(performance)

KeyboardInterrupt: 

In [ ]:
X_train_ss, X_test_ss = cmpre.ss_scale(X_train, X_test)

smote = SMOTE(random_state = 15)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ss, y_train)
    
gb = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)
gb.fit(X_train_resampled, y_train_resampled)

print(f'Train: {recall_score(y_train, gb.predict(X_train_ss))}')
print(f'Test:  {recall_score(y_test, gb.predict(X_test_ss))}')

# MODEL 3B: GBC w/ ADASYN Balance

- model:
    - GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)
- scaling: StandardScaler()
- balancing: ADASYN

In [23]:
fold_count=0
for train_ind, val_ind in kf.split(X_train, y_train):
    fold_count+=1

    performance = pd.DataFrame(columns=['Train_Recall','Test_Recall','Test_Specificity'])

    recall = make_scorer(recall_score)

    X_t, y_t = X_train.iloc[train_ind], y_train.iloc[train_ind]
    X_val, y_val = X_train.iloc[val_ind], y_train.iloc[val_ind]
    
    # instantiate and fit/transform scaler
    X_t_sc, X_val_sc = cmpre.ss_scale(X_t, X_val)

    # instantiate and fit ADASYN: Oversample minority
    X_t_sc_bal, y_t_bal = ADASYN(sampling_strategy='minority',random_state=42).fit_resample(X_t_sc, y_t)
    
    
    clf = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        random_state = 15)

    train_cv = cross_val_score(X=X_t_sc_bal, y=y_t_bal, estimator=clf, scoring=recall,cv=10)

    # Predict
    y_pred = clf.fit(X_t_sc_bal, y_t_bal).predict(X_val_sc)

    conf_matrix = confusion_matrix(y_val,y_pred)

    # Store results
    performance.loc['GBC_ADASYN',
                    ['Train_Recall','Test_Recall','Test_Specificity']] = [
        train_cv.mean(),
        recall_score(y_val,y_pred),
        conf_matrix[0,0]/conf_matrix[0,:].sum()
    ]
    print(performance)

           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN     0.832246        0.85         0.857143
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN      0.84551    0.853333         0.884706
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN     0.832084    0.838235          0.87037
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN     0.832845         0.9         0.860465
           Train_Recall Test_Recall Test_Specificity
GBC_ADASYN      0.82503    0.876923          0.87788


In [24]:
X_train_ss, X_test_ss = cmpre.ss_scale(X_train, X_test)

X_train_resampled, y_train_resampled = ADASYN(sampling_strategy='minority',random_state=42).fit_resample(X_train_ss, y_train)
    
gb = GradientBoostingClassifier(
        loss = 'exponential', 
        learning_rate = 0.01,
        min_samples_leaf = 0.1,
        min_samples_split = 0.13636363636363638,
        max_depth = 3,
        criterion = 'mae',
        random_state = 15)
gb.fit(X_train_resampled, y_train_resampled)

print(f'Train: {recall_score(y_train, gb.predict(X_train_ss))}')
print(f'Test:  {recall_score(y_test, gb.predict(X_test_ss))}')

Train: 0.8631284916201117
Test:  0.888
